# Manuscript Companion for: "Mapping Hippocampal and Thalamic Atrophy in Epilepsy: A 7T MRI Study"

## Load in data

In [42]:
import pandas as pd
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt

with open(r'subfield_volumes_by_pt.pickle', "rb") as input_file:
    e = pickle.load(input_file)
    seg_vol = e['seg_vol']
    vol_pts = e['vol_pts']
    fmri_lab= e['vol_lab']

pt_info = pd.read_excel('/Users/smouc/LittLab/Alfredo/Segmentation/Code/hippo_thal_7t_metadata_updated_5SENSE.xlsx').sort_values(by='Subject')

In [43]:
group_assignments = np.array(list(map(lambda x: int(pt_info[pt_info['Subject']==x]['Lat'].values[0] == 'Control'), vol_pts)))
protocol_versions = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['version'].values[0], vol_pts)))

from neuroCombat import neuroCombat

# Specifying the batch (scanner variable) as well as a biological covariate to preserve:
covars = {'batch':protocol_versions,
          'group':group_assignments} 
covars = pd.DataFrame(covars)

# To specify names of the variables that are categorical:
categorical_cols = ['group']

# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col = 'batch'

#Harmonization step:
data_combat = neuroCombat(dat=seg_vol, # should be (samples, features)
    covars=covars,
    batch_col=batch_col,
    categorical_cols=categorical_cols)["data"]

seg_vol = data_combat.T
print(seg_vol.shape)

[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
(55, 24)


In [44]:
ep_vol_pts = [x for x in vol_pts if (pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] != 'Control')]

temporal = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] == 'Temporal', ep_vol_pts)))
frontal = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] == 'Frontal', ep_vol_pts)))
control = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] == 'Control', ep_vol_pts)))
left = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['Lat'].values[0] == 'L', ep_vol_pts)))
right = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['Lat'].values[0] == 'R', ep_vol_pts)))
bilateral = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['Lat'].values[0] == 'B', ep_vol_pts)))
lesional = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['MRI_Lesional'].values[0] == 'Lesional', ep_vol_pts)))
nonlesional = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['MRI_Lesional'].values[0] == 'Nonlesional', ep_vol_pts)))
mts = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['MTS (y/n)'].values[0] == 'y', ep_vol_pts)))
nonmts = np.array(list(map(lambda x: pt_info[pt_info['Subject']==x]['MTS (y/n)'].values[0] == 'n', ep_vol_pts)))

seg_vol_ep = seg_vol[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] != 'Control' for x in vol_pts], :]
seg_vol_hc = seg_vol[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] == 'Control' for x in vol_pts], :]

# Build LDA Model

In [28]:
from sklearn.decomposition import LatentDirichletAllocation
from scipy import stats
import matplotlib.pyplot as plt

def gen_save_lda(seg_vol, groups, save_name):
# groups = np.array(list(map(lambda x: int(pt_info[pt_info['Subject']==x]['Lat'].values[0] == 'Control'), vol_pts)))

# seg_vol = patients x structures
# ep seg_vol = seg_vol[group_assignments == 0, :]; control seg_vol = seg_vol[group_assignments == 1, :]

    k=4
    numrun = 10
    numi = 100
    dup = 0

    ep_seg_vol = seg_vol[groups == 0, :] # epilepsy
    c_seg_vol = seg_vol[groups == 1, :] # control

    # z-score and convert to LDA compatible dataset
    ep_seg_vol_norm = np.abs((ep_seg_vol - np.mean(c_seg_vol,axis=0))/np.std(c_seg_vol,axis=0)*10).astype(int)

    seg_vol_norm_left = ep_seg_vol_norm[left,:]
    seg_vol_norm_right = ep_seg_vol_norm[right,:]

    ind = np.hstack([np.arange(4, 8), np.arange(0, 4), np.arange(16, 24), np.arange(8, 16)])
    seg_vol_norm_right_flipped = seg_vol_norm_right[:, ind]

    ep_seg_vol_norm_all = np.vstack([seg_vol_norm_left, seg_vol_norm_right_flipped])
    
    best_of_best_model = []
    best_of_best_comp = []

    for run in range(numrun):
        out_components = []
        out_models = []

        # generate 100 models
        i=0
        while i <= numi:
            lda = LatentDirichletAllocation(n_components=k)
            lda.fit(ep_seg_vol_norm)

            if (np.min(np.std(lda.components_ / np.tile(np.sum(lda.components_, axis=1), (24,1)).T, axis=1)) >= 10**-4):
                out_components.append(lda.components_)
                out_models.append(lda)
                i+=1
            else:
                dup+=1

        comp_corr = np.zeros((len(out_models),len(out_models)))

        # iterate through every pair of models and find the largest correlation between any pair of factors
        for i in range(len(out_models)):
            for j in range(i+1, len(out_models)):
                i_comp = out_components[i]/out_components[i].sum(axis=1)[:, np.newaxis]
                j_comp = out_components[j]/out_components[j].sum(axis=1)[:, np.newaxis]

                p_all = []
                for f1 in range(k):
                    for f2 in range(k):
                        p_all.append(stats.pearsonr(i_comp[f1,:], j_comp[f2,:]))

                comp_corr[i,j] = np.max(p_all)

        comp_corr += comp_corr.T
        comp_corr += np.eye(len(out_models))

        avg_corr = np.mean(comp_corr, axis=1)
        best_run = np.argmax(avg_corr)

        best_of_best_model.append(out_models[best_run])
        best_of_best_comp.append(out_components[best_run])

    best_comp_corr = np.zeros((numrun,numrun))

    for i in range(numrun):
        for j in range(i+1, numrun):
            i_comp = best_of_best_comp[i]/best_of_best_comp[i].sum(axis=1)[:, np.newaxis]
            j_comp = best_of_best_comp[j]/best_of_best_comp[j].sum(axis=1)[:, np.newaxis]

            p_all = []
            for f1 in range(k):
                for f2 in range(k):
                    p_all.append(stats.pearsonr(i_comp[f1,:], j_comp[f2,:]))

            best_comp_corr[i,j] = np.max(p_all)

    best_comp_corr += best_comp_corr.T
    best_comp_corr += np.eye(numrun)

    avg_corr = np.mean(best_comp_corr, axis=1)
    best_run = np.argmax(avg_corr)
    lda_components_new = best_of_best_comp[best_run]/best_of_best_comp[best_run].sum(axis=1)[:, np.newaxis]
    
    finalout_factors = best_of_best_model[best_run].transform(ep_seg_vol_norm_all)

    with open(save_name, 'wb') as output_file:
        pickle.dump({'lda_components':lda_components_new, 'lda_model':best_of_best_model[best_run], 'pt_factors':finalout_factors}, output_file)

    print(dup)
    return(finalout_factors)

In [ ]:
groups = np.array(list(map(lambda x: int(pt_info[pt_info['Subject']==x]['Lat'].values[0] == 'Control'), vol_pts)))
out = gen_save_lda(seg_vol, groups, 'lda_components.pickle')

## Figure 2 - Comparison of Hippocampal and Thalamic volumes in epilepsy subgroups vs controls

In [ ]:
import numpy as np
import pyvista as pv
from scipy import stats
import seaborn as sns
pv.global_theme.cmap = 'RdBu'

def plot_ashs_no_tail(vals, vmin, vmax):
    # Load the Left Hippocampus
    l_ca1 = pv.read('../../../source_data/surfaces/hippocampus/hippocampus00001.stl')
    l_ca2 = pv.read('../../../source_data/surfaces/hippocampus/hippocampus00002.stl')
    l_dg = pv.read('../../../source_data/surfaces/hippocampus/hippocampus00003.stl')
    l_ca3 = pv.read('../../../source_data/surfaces/hippocampus/hippocampus00004.stl')

    l_ca1.point_data['Segmentation'] = vals[0]
    l_ca2.point_data['Segmentation'] = vals[1]
    l_dg.point_data['Segmentation'] = vals[2]
    l_ca3.point_data['Segmentation'] = vals[3]

    hippocampus = l_ca1.merge([l_ca1, l_ca2, l_dg, l_ca3])


    # Load the Right Hippocampus
    r_ca1 = pv.read('../../../source_data/surfaces/hippocampus/hippocampus00101.stl')
    r_ca2 = pv.read('../../../source_data/surfaces/hippocampus/hippocampus00102.stl')
    r_dg = pv.read('../../../source_data/surfaces/hippocampus/hippocampus00103.stl')
    r_ca3 = pv.read('../../../source_data/surfaces/hippocampus/hippocampus00104.stl')

    r_ca1.point_data['Segmentation'] = vals[4]
    r_ca2.point_data['Segmentation'] = vals[5]
    r_dg.point_data['Segmentation'] = vals[6]
    r_ca3.point_data['Segmentation'] = vals[7]

    hippocampus = hippocampus.merge([r_ca1, r_ca2, r_dg, r_ca3])

    # Do the actual plotting
    # Hippocampus Camera Position
    hipp_cpos = [(-9.979561975549055, 10.84928567303749, 123.90691105330777), (-0.04879951477050781, 14.638345837593079, -15.511809825897217), (0.06734247854283312, 0.9972198643766981, 0.03189878800563263)]


    p = pv.Plotter()
    p.add_mesh(hippocampus)
    p.update_scalar_bar_range([vmin, vmax])

    p.show(jupyter_backend='static', cpos=hipp_cpos)

def plot_ashs_no_ca23(vals, vmin, vmax):
    # Load the Left Hippocampus
    l_ca1 = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00001.stl')
    l_ca2 = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00002.stl')
    l_dg = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00003.stl')
    l_ca3 = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00004.stl')
    l_tail = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00005.stl')

    l_ca1.point_data['Segmentation'] = vals[0]
    
    l_dg.point_data['Segmentation'] = vals[2]

    l_tail.point_data['Segmentation'] = vals[4]

    hippocampus = l_ca1.merge([l_ca1, l_dg, l_tail])


    # Load the Right Hippocampus
    r_ca1 = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00101.stl')
    r_ca2 = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00102.stl')
    r_dg = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00103.stl')
    r_ca3 = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00104.stl')
    r_tail = pv.read('/Users/allucas/Documents/research/CNT/P36_hippo_thal_7t/source_data/surfaces/hippocampus/hippocampus00105.stl')

    r_ca1.point_data['Segmentation'] = vals[5]
    r_dg.point_data['Segmentation'] = vals[7]
    r_tail.point_data['Segmentation'] = vals[9]

    hippocampus = hippocampus.merge([r_ca1, r_dg, r_tail])

    # Do the actual plotting
    # Hippocampus Camera Position
    hipp_cpos = [(-9.979561975549055, 10.84928567303749, 123.90691105330777), (-0.04879951477050781, 14.638345837593079, -15.511809825897217), (0.06734247854283312, 0.9972198643766981, 0.03189878800563263)]


    p = pv.Plotter()
    p.add_mesh(hippocampus)
    p.update_scalar_bar_range([vmin, vmax])

    p.show(jupyter_backend='static', cpos=hipp_cpos)

def plot_thomas(vals, vmin, vmax):
    # Load the Left Thalamus
    l_av = pv.read('../../../source_data/surfaces/thalamus/thalamus00002.stl')
    l_va = pv.read('../../../source_data/surfaces/thalamus/thalamus00004.stl')
    l_vla = pv.read('../../../source_data/surfaces/thalamus/thalamus00005.stl')
    l_vlp = pv.read('../../../source_data/surfaces/thalamus/thalamus00006.stl')
    l_vpl = pv.read('../../../source_data/surfaces/thalamus/thalamus00007.stl')
    l_pul = pv.read('../../../source_data/surfaces/thalamus/thalamus00008.stl')
    l_cm = pv.read('../../../source_data/surfaces/thalamus/thalamus00011.stl')
    l_md = pv.read('../../../source_data/surfaces/thalamus/thalamus00012.stl')


    l_av.point_data['Segmentation'] = vals[0]
    l_va.point_data['Segmentation'] = vals[1]
    l_vla.point_data['Segmentation'] = vals[2]
    l_vlp.point_data['Segmentation'] = vals[3]
    l_vpl.point_data['Segmentation'] = vals[4]
    l_pul.point_data['Segmentation'] = vals[5]
    l_cm.point_data['Segmentation'] = vals[6]
    l_md.point_data['Segmentation'] = vals[7]

    thalamus_l = l_av.merge([l_va, l_vla, l_vlp, l_vpl, l_pul, l_cm, l_md])


    # Load the Right Thalamus
    r_av = pv.read('../../../source_data/surfaces/thalamus/thalamus00102.stl')
    r_va = pv.read('../../../source_data/surfaces/thalamus/thalamus00104.stl')
    r_vla = pv.read('../../../source_data/surfaces/thalamus/thalamus00105.stl')
    r_vlp = pv.read('../../../source_data/surfaces/thalamus/thalamus00106.stl')
    r_vpl = pv.read('../../../source_data/surfaces/thalamus/thalamus00107.stl')
    r_pul = pv.read('../../../source_data/surfaces/thalamus/thalamus00108.stl')
    r_cm = pv.read('../../../source_data/surfaces/thalamus/thalamus00111.stl')
    r_md = pv.read('../../../source_data/surfaces/thalamus/thalamus00112.stl')


    r_av.point_data['Segmentation'] = vals[8]
    r_va.point_data['Segmentation'] = vals[9]
    r_vla.point_data['Segmentation'] = vals[10]
    r_vlp.point_data['Segmentation'] = vals[11]
    r_vpl.point_data['Segmentation'] = vals[12]
    r_pul.point_data['Segmentation'] = vals[13]
    r_cm.point_data['Segmentation'] = vals[14]
    r_md.point_data['Segmentation'] = vals[15]

    thalamus_r = r_av.merge([r_av, r_va, r_vla, r_vlp, r_vpl, r_pul, r_cm, r_md])

    thalamus = thalamus_l.merge(thalamus_r)


    thalamus_l_position = [(-94.93243156636869, 20.80048340483919, -3.6609767056495),(0.3974123737947691, 17.327023074415486, -8.203546407573794), (0.04772021646463825, 0.003395016921239654, 0.9988549718556106)]

    thalamus_r_position = [(95.60832355891446, 12.133001374531405, -13.531167049111657), (0.3974123737947691, 17.327023074415486, -8.203546407573794), (0.059400106557700456, 0.06727258338579319, 0.9959648723050103)]
    thalamus_position = [(11.081681826398091, 14.736747032236908, 123.86774195231541), (-0.04879951477050781, 14.638345837593079, -15.511809825897217), (0.02417376168990972, 0.9997042960265031, -0.0026362382126156984)]


    p = pv.Plotter()
    p.add_mesh(thalamus)
    p.update_scalar_bar_range([vmin, vmax])

    p.show(jupyter_backend='static', cpos=thalamus_position)

In [10]:
ind = np.hstack([np.arange(4, 8), np.arange(0, 4), np.arange(16, 24), np.arange(8, 16)])

seg_vol_left = seg_vol_ep[left,:]
seg_vol_right = seg_vol_ep[right,:]
seg_vol_right_flipped = seg_vol_right[:, ind]
seg_vol_ep_ipsi = np.vstack([seg_vol_left, seg_vol_right_flipped])

seg_vol_ep_zscore = (seg_vol_ep - np.mean(seg_vol_hc,axis=0))/np.std(seg_vol_hc,axis=0)

seg_vol_zscore_left = seg_vol_ep_zscore[left,:]
seg_vol_zscore_right = seg_vol_ep_zscore[right,:]

seg_vol_zscore_right_flipped = seg_vol_zscore_right[:, ind]

seg_vol_ep_ipsi_zscore = np.vstack([seg_vol_zscore_left, seg_vol_zscore_right_flipped])

mts_lr = np.hstack([mts[left], mts[right]])
nonmts_lr = np.hstack([nonmts[left], nonmts[right]])
temporal_lr = np.hstack([temporal[left], temporal[right]])
frontal_lr = np.hstack([frontal[left], frontal[right]])
left_lr = np.hstack([left[left], left[right]])
right_lr = np.hstack([right[left], right[right]])
lesional_lr = np.hstack([lesional[left], lesional[right]])
nonlesional_lr = np.hstack([nonlesional[left], nonlesional[right]])

In [11]:
from scipy import stats
from statsmodels.stats.multitest import multipletests
import seaborn as sns
from numpy import mean, std # version >= 1.7.1 && <= 1.9.1
from math import sqrt

def cohen_d(x,y):
    return (mean(x) - mean(y)) / sqrt((std(x, ddof=1) ** 2 + std(y, ddof=1) ** 2) / 2.0)

def cohen_d_mat(x,y):
    return (np.mean(x, axis = 0) - np.mean(y, axis = 0)) / np.sqrt((np.std(x, ddof=1, axis=0) ** 2 + np.std(y, ddof=1, axis=0) ** 2) / 2.0)

def z_score(x,y):
    return (x-np.mean(y))/np.std(y)

ipsi_fmri_lab = [x.replace('L-', 'Ipsi-').replace('R-', 'Contra-') for x in fmri_lab]

outfolder = '/Users/smouc/LittLab/Alfredo/Segmentation/Code/figs/'

def fig2_rawvolume_plot(g1, g2, title, lab1, lab2):
    plt.figure(figsize=(8,4))
    ax = plt.subplot()
    plt.bar(np.arange(24), np.mean(seg_vol_ep_ipsi[g1,:], axis=0), width = 0.35, yerr = np.std(seg_vol_ep_ipsi[g1,:], axis=0), color = 'b', label = lab1)
    plt.bar(np.arange(24)+0.35, np.mean(seg_vol_ep_ipsi[g2,:], axis=0), width = 0.35, yerr = np.std(seg_vol_ep_ipsi[g2,:], axis=0), color = 'r', label = lab2)
    ax.set_xticks(np.arange(24))
    ax.set_xticklabels(ipsi_fmri_lab, rotation = 75)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend(loc='upper right', frameon=False)
    # ax.spines['bottom'].set_visible(False)
    
    pvals_g1g2 = []
    pvals_g1ref = []
    pvals_g2ref = []
    
    for i in range(24):
        pvals_g1g2.append(stats.ttest_ind(seg_vol_ep_ipsi_zscore[g1,i], seg_vol_ep_ipsi_zscore[g2,i])[1])
        #pvals_g1ref.append(stats.ttest_ind(seg_vol_ep_ipsi[g1,i], seg_vol_hc[:,i])[1])
        #pvals_g2ref.append(stats.ttest_ind(seg_vol_ep_ipsi[g2,i], seg_vol_hc[:,i])[1])
        pvals_g1ref.append(stats.ttest_1samp(seg_vol_ep_ipsi_zscore[g1,i], popmean=0)[1])
        pvals_g2ref.append(stats.ttest_1samp(seg_vol_ep_ipsi_zscore[g2,i], popmean=0)[1])
    pvals_g1g2bh = multipletests(pvals_g1g2, alpha=0.05, method='fdr_bh')
    pvals_g1refbh = multipletests(pvals_g1ref, alpha=0.05, method='fdr_bh')
    pvals_g2refbh = multipletests(pvals_g2ref, alpha=0.05, method='fdr_bh')
    
    print('Group 1')
    for i,p in enumerate(pvals_g1refbh[1]):
        print(ipsi_fmri_lab[i], ': ', p,'\n')

    print('Group 2')
    for i,p in enumerate(pvals_g2refbh[1]):
        print(ipsi_fmri_lab[i], ': ', p,'\n')
    
    # print(pvals_g1refbh[0])
    # print(pvals_g2refbh[0])
    
    star_left_ind = np.arange(24) - .04
    star_right_ind = star_left_ind + .28
    
    plt.scatter(star_left_ind[pvals_g1refbh[0]], (np.mean(seg_vol_ep_ipsi[g1,:], axis=0) + np.std(seg_vol_ep_ipsi[g1,:], axis=0) + 200)[pvals_g1refbh[0]], color = 'k', marker = "*")
    plt.scatter(star_right_ind[pvals_g2refbh[0]], (np.mean(seg_vol_ep_ipsi[g2,:], axis=0) + np.std(seg_vol_ep_ipsi[g2,:], axis=0) + 200)[pvals_g2refbh[0]], color = 'k', marker = "*")
    plt.ylim([0, 5000])
    
    #plt.savefig(outfolder + 'ttestvolume_figure_%s.svg'%title, format='svg', bbox_inches="tight")
    plt.show()
    
    return cohen_d_mat(seg_vol_ep_ipsi_zscore[g1,:], seg_vol_ep_ipsi_zscore[g2,:])


def fig2_cohensd_plot(g1, g2, title, lab1, lab2):
    plt.figure(figsize=(14,5))
    cohend_g1 = []
    cohend_g2 = []
    for i in range(24):
        cohend_g1.append(cohen_d_mat(seg_vol_ep_ipsi[g1,i], seg_vol_hc[:,i]))
        cohend_g2.append(cohen_d_mat(seg_vol_ep_ipsi[g2,i], seg_vol_hc[:,i]))
    
    df_plot = pd.DataFrame()
    df_plot["Cohen's d"] = cohend_g1 + cohend_g2
    df_plot["Group"] = [lab1]*24 + [lab2]*24
    df_plot["Region"] = ipsi_fmri_lab*2
    sns.barplot(x='Region', y="Cohen's d", hue='Group', data=df_plot, linewidth=0.5, edgecolor='k', width=0.5)
    plt.hlines(y=0, xmin=-0.5, xmax=23.5, color='k')
    sns.despine()
    plt.xticks(rotation=75)
    # pyvista plot for g1
    plot_ashs_no_tail(cohend_g1[:8],vmin=-4,vmax=4)
    plot_thomas(cohend_g1[8:],vmin=-2,vmax=2)

    # pyvista plot for g2
    plot_ashs_no_tail(cohend_g2[:8],vmin=-4,vmax=4)
    plot_thomas(cohend_g2[8:],vmin=-2,vmax=2)

def fig2_zscore_plot(g1, g2, title, lab1, lab2):
    plt.figure(figsize=(10,5))
    df_plot_all = pd.DataFrame()
    seg_g1 = seg_vol_ep_ipsi_zscore[g1,:]
    for i in range(seg_g1.shape[0]):
        zscore_g1 = []
        for j in range(24):
            zscore_g1.append(seg_g1[i,j])
        
        df_plot = pd.DataFrame()
        df_plot["Z-Score"] = zscore_g1
        df_plot["Group"] = [lab1]*24
        df_plot["Region"] = ipsi_fmri_lab

        df_plot_all = pd.concat((df_plot_all, df_plot))

    seg_g2 = seg_vol_ep_ipsi_zscore[g2,:]
    for i in range(seg_g2.shape[0]):
        zscore_g2 = []
        for j in range(24):
            zscore_g2.append(seg_g2[i,j])
        
        df_plot = pd.DataFrame()
        df_plot["Z-Score"] = zscore_g2
        df_plot["Group"] = [lab2]*24
        df_plot["Region"] = ipsi_fmri_lab

        df_plot_all = pd.concat((df_plot_all, df_plot))


    sns.barplot(x='Region', y="Z-Score", hue='Group', data=df_plot_all, linewidth=0.5, edgecolor='k')
    sns.despine()
    plt.hlines(y=0, xmin=-0.5, xmax=23.5, color='k')
    plt.xticks(rotation=75)

def fig2_rawvolume2_plot(g1, g2, title, lab1, lab2):
    plt.figure(figsize=(10,5))
    df_plot_all = pd.DataFrame()
    seg_g1 = seg_vol_ep_ipsi[g1,:]
    for i in range(seg_g1.shape[0]):
        zscore_g1 = []
        for j in range(24):
            zscore_g1.append(seg_g1[i,j])
        
        df_plot = pd.DataFrame()
        df_plot["Volume"] = zscore_g1
        df_plot["Group"] = [lab1]*24
        df_plot["Region"] = ipsi_fmri_lab

        df_plot_all = pd.concat((df_plot_all, df_plot))

    seg_g2 = seg_vol_ep_ipsi[g2,:]
    for i in range(seg_g2.shape[0]):
        zscore_g2 = []
        for j in range(24):
            zscore_g2.append(seg_g2[i,j])
        
        df_plot = pd.DataFrame()
        df_plot["Volume"] = zscore_g2
        df_plot["Group"] = [lab2]*24
        df_plot["Region"] = ipsi_fmri_lab

        df_plot_all = pd.concat((df_plot_all, df_plot))


    seg_g3 = seg_vol_hc
    for i in range(seg_g3.shape[0]):
        zscore_g3 = []
        for j in range(24):
            zscore_g3.append(seg_g3[i,j])
        
        df_plot = pd.DataFrame()
        df_plot["Volume"] = zscore_g3
        df_plot["Group"] = ['Control']*24
        df_plot["Region"] = ipsi_fmri_lab

        df_plot_all = pd.concat((df_plot_all, df_plot))

    sns.barplot(x='Region', y="Volume", hue='Group', data=df_plot_all, linewidth=0.5, edgecolor='k')
    sns.despine()
    plt.hlines(y=0, xmin=-0.5, xmax=23.5, color='k')
    plt.xticks(rotation=75)


def fig2_zscore_volume_single_plot(g1,  title, lab1):
    plt.figure(figsize=(10,6))
    df_plot_all = pd.DataFrame()
    seg_g1 = seg_vol_ep_ipsi_zscore[g1,:]
    for i in range(seg_g1.shape[0]):
        zscore_g1 = []
        for j in range(24):
            zscore_g1.append(seg_g1[i,j])
        
        df_plot = pd.DataFrame()
        df_plot["Volume"] = zscore_g1
        df_plot["Group"] = [lab1]*24
        df_plot["Region"] = ipsi_fmri_lab
        df_plot["Structure"] = ['ipsi-Hippocampus']*4 + ['contra-Hippocampus']*4 + ['ipsi-Thalamus']*8 + ['contra-Thalamus']*8
        df_plot_all = pd.concat((df_plot_all, df_plot))


    sns.barplot(x='Region', y="Volume", hue='Structure', data=df_plot_all, linewidth=1, edgecolor='k', dodge=False)
    sns.despine()
    plt.hlines(y=0, xmin=-0.5, xmax=23.5, color='k')
    plt.xticks(rotation=75)
    plt.xlabel('Region',fontweight='bold')
    plt.ylabel('Volume',fontweight='bold')
    plt.tight_layout()
    # plt.savefig('/Volumes/T7_Shield/research/CNT/P36_hippo_thal_7t/outputs/figures/paper_figures/jupyter_figures/'+lab1+'_raw_volumes.pdf')

In [ ]:
sns.set_palette('Spectral')
print('MTS vs nonMTS')
cohend_mts_nonmts = fig2_rawvolume2_plot(mts_lr, nonmts_lr, 'mts', 'MTS', 'No MTS')
fig2_cohensd_plot(mts_lr, nonmts_lr, 'mts', 'MTS', 'No MTS')
fig2_zscore_plot(mts_lr, nonmts_lr, 'mts', 'MTS', 'No MTS')

print('Left Temporal vs Right Temporal')
cohend_ltle_rtle = fig2_rawvolume2_plot(left_lr * temporal_lr, right_lr * temporal_lr, 'ltle', 'LTLE', 'RTLE')
fig2_cohensd_plot(left_lr * temporal_lr, right_lr * temporal_lr, 'ltle', 'LTLE', 'RTLE')
fig2_zscore_plot(left_lr * temporal_lr, right_lr * temporal_lr, 'ltle', 'LTLE', 'RTLE')

print('Temporal vs Frontal')
cohend_temp_frontal = fig2_rawvolume2_plot(temporal_lr, frontal_lr, 'temp', 'Temporal', 'Frontal')
fig2_cohensd_plot(temporal_lr, frontal_lr, 'temp', 'Temporal', 'Frontal')
fig2_zscore_plot(temporal_lr, frontal_lr, 'temp', 'Temporal', 'Frontal')

print('Lesional vs non lesional')
cohend_les_nonles = fig2_rawvolume2_plot(lesional_lr, nonlesional_lr, 'les', 'Lesional', 'Nonlesional')
fig2_cohensd_plot(lesional_lr, nonlesional_lr, 'les', 'Lesional', 'Nonlesional')
fig2_zscore_plot(lesional_lr, nonlesional_lr, 'les', 'Lesional', 'Nonlesional')

## Figure 3 - Hippocampal and Thalamic Atrophy Factors

In [54]:
with open(r'lda_components.pickle', 'rb') as f:
    # left hippocampus first column, right hippocampus second column
    file = pickle.load(f)
    lda_comp = file['lda_components']
    lda_mod = file['lda_model']

print(lda_comp.shape)

seg_vol_ep = seg_vol[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] != 'Control' for x in vol_pts], :]
seg_vol_hc = seg_vol[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] == 'Control' for x in vol_pts], :]

seg_vol_ep_norm = np.abs((seg_vol_ep - np.mean(seg_vol_hc,axis=0))/np.std(seg_vol_hc,axis=0)*10).astype(int)

seg_vol_norm_left = seg_vol_ep_norm[left,:]
seg_vol_norm_right = seg_vol_ep_norm[right,:]

ind = np.hstack([np.arange(4, 8), np.arange(0, 4), np.arange(16, 24), np.arange(8, 16)])
seg_vol_norm_right_flipped = seg_vol_norm_right[:, ind]

ep_seg_vol_norm = np.vstack([seg_vol_norm_left, seg_vol_norm_right_flipped])
ep_seg_vol_factors = lda_mod.transform(ep_seg_vol_norm)
ep_seg_vol_norm_lr = np.vstack([seg_vol_norm_left, seg_vol_norm_right])
ep_seg_vol_factors_lr = lda_mod.transform(ep_seg_vol_norm_lr)

print(ep_seg_vol_factors.shape)

(4, 24)
(33, 4)


In [ ]:
# plot the LDA components
lda = lda_mod
for i in range(4):
    component = i
    print('Factor ', i+1)
    lda_components = lda.components_/ lda.components_.sum(axis=1)[:, np.newaxis]
    plot_ashs_no_tail(lda_components[component,:8], 0, 0.07)
    plot_thomas(lda_components[component,8:], 0, 0.07)
    # plt.bar(x=np.arange(len(ashs_regions_to_gather)+len(thomas_regions_to_gather)),height=lda_components[component,:])
    # plt.xticks(np.arange(len(ashs_regions_to_gather)+len(thomas_regions_to_gather)), ashs_regions_to_gather+thomas_regions_to_gather, rotation=90)

    df_plot = pd.DataFrame()
    df_plot['Component Weight'] = lda_components[component,:]
    df_plot['Component Name'] =  ipsi_fmri_lab

    df_plot['Region'] = ['I-Hippocampus']*4 + ['C-Hippocampus']*4 + ['I-Thalamus']*8 + ['C-Thalamus']*8

    sns.set_palette(['#9D291E', '#FB8B24','#1E929D','#143747'])

    plt.figure(figsize=(10,5))
    plt.subplot(121)
    sns.barplot(data=df_plot, x='Component Name', y='Component Weight', hue='Region' ,order=df_plot['Component Name'].values[np.argsort(lda_components[component,:])[::-1]],
                edgecolor='k', dodge=False, saturation=1)
    plt.xticks(rotation=90)
    plt.subplot(122)
    sns.barplot(data=df_plot, x='Region', y='Component Weight', edgecolor='k', linewidth=3, capsize=.4, errcolor="0", saturation=1)
    plt.xticks(rotation=90)
    sns.despine()
    plt.tight_layout()
    #plt.savefig('/Volumes/T7_Shield/research/CNT/P36_hippo_thal_7t/outputs/figures/paper_figures/jupyter_figures/factor'+str(i+1)+'.pdf')

## Figure 4 - Distribution of disease factors across different epilepsy subtypes

In [56]:
def plot_factor_heatmap(g1,lab):
    sorted1 = np.argsort(ep_seg_vol_factors[g1][:,0])
    sorted2 = np.argsort(ep_seg_vol_factors[g1][:,1])
    sorted3 = np.argsort(ep_seg_vol_factors[g1][:,2])
    sorted4 = np.argsort(ep_seg_vol_factors[g1][:,3])
    plt.subplot(141)
    plt.imshow(ep_seg_vol_factors[g1][sorted1,:], cmap='inferno')
    plt.subplot(142)
    plt.imshow(ep_seg_vol_factors[g1][sorted2,:], cmap='inferno')
    plt.subplot(143)
    plt.imshow(ep_seg_vol_factors[g1][sorted3,:], cmap='inferno')
    plt.subplot(144)
    plt.imshow(ep_seg_vol_factors[g1][sorted4,:], cmap='inferno')
    plt.colorbar()
    # plt.savefig(os.path.join('/Volumes/T7_Shield/research/CNT/P36_hippo_thal_7t/outputs/figures/paper_figures/jupyter_figures','factor_heatmatp_'+lab+'.pdf'))
    plt.figure()

    print('Maximum factors: ',np.argmax(ep_seg_vol_factors[g1],axis=1)+1)
    spearman_matrix = np.zeros((4,4))
    spearman_pvalues = np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            spearman_matrix[i,j] = stats.spearmanr(ep_seg_vol_factors[g1][:,i],ep_seg_vol_factors[g1][:,j])[0]
            spearman_pvalues[i,j] = stats.spearmanr(ep_seg_vol_factors[g1][:,i],ep_seg_vol_factors[g1][:,j])[1]

    spearman_matrix[np.triu_indices(4,0)] = np.nan
    print(spearman_matrix)
    print('Significant cells: ',(spearman_pvalues*6))
    # Create the heatmap plot
    sns.set_style("white")
    ax = sns.heatmap(spearman_matrix, cmap='RdBu', vmin=-1, vmax=1, annot=True,
                    fmt='.2f', annot_kws={"size": 14}, linewidths=.5,
                    xticklabels=['Factor 1', 'Factor 2', 'Factor 3'],
                    yticklabels=['', 'Factor 2', 'Factor 3', 'Factor 4'])
    # plt.savefig(os.path.join('/Volumes/T7_Shield/research/CNT/P36_hippo_thal_7t/outputs/figures/paper_figures/jupyter_figures','factor_cofluctuation_'+lab+'.pdf'))
    plt.show()

In [ ]:
plot_factor_heatmap(mts_lr,'mts')
plt.figure()
plot_factor_heatmap(nonmts_lr,'non-mts')

In [ ]:
plot_factor_heatmap(left_lr*temporal_lr, 'LTLE')
plt.figure()
plot_factor_heatmap(right_lr*temporal_lr, 'RTLE')

5 sense scores

In [61]:
fivesense_vals = list(map(lambda x: pt_info[pt_info['Subject']==x]['fivesense'].values[0] if x in list(pt_info.Subject) else np.nan, vol_pts))
patient_vals = list(map(lambda x: pt_info[pt_info['Subject']==x]['Subject'].values[0] if x in list(pt_info.Subject) else np.nan, vol_pts))
loc_vals = list(map(lambda x: pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] if x in list(pt_info.Subject) else np.nan, vol_pts))
outcome_vals = list(map(lambda x: pt_info[pt_info['Subject']==x]['Outcome'].values[0] if x in list(pt_info.Subject) else np.nan, vol_pts))
lat_vals = list(map(lambda x: pt_info[pt_info['Subject']==x]['Lat'].values[0] if x in list(pt_info.Subject) else np.nan, vol_pts))
mts_vals = list(map(lambda x: pt_info[pt_info['Subject']==x]['MTS (y/n)'].values[0] if x in list(pt_info.Subject) else np.nan, vol_pts))

fivesense_vals = np.array(fivesense_vals)[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] != 'Control' for x in vol_pts]]
fivesense_vals = np.hstack((fivesense_vals[left], fivesense_vals[right]))

patient_vals = np.array(patient_vals)[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] != 'Control' for x in vol_pts]]
patient_vals = np.hstack((patient_vals[left], patient_vals[right]))

loc_vals = np.array(loc_vals)[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] != 'Control' for x in vol_pts]]
loc_vals = np.hstack((loc_vals[left], loc_vals[right]))

outcome_vals = np.array(outcome_vals)[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] != 'Control' for x in vol_pts]]
outcome_vals = np.hstack((outcome_vals[left], outcome_vals[right]))

lat_vals = np.array(lat_vals)[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] != 'Control' for x in vol_pts]]
lat_vals = np.hstack((lat_vals[left], lat_vals[right]))

mts_vals = np.array(mts_vals)[[pt_info[pt_info['Subject']==x]['Final_Loc'].values[0] != 'Control' for x in vol_pts]]
mts_vals = np.hstack((mts_vals[left], mts_vals[right]))

In [ ]:
df_plot = pd.DataFrame()
df_plot['Fivesense'] =  fivesense_vals
df_plot['Subject'] = patient_vals
df_plot['Final Location'] = loc_vals
df_plot['Good Outcome'] = outcome_vals
df_plot['Lat'] = lat_vals
df_plot['MTS'] = mts_vals
df_plot['Factor 3'] = ep_seg_vol_factors[:,2]
df_plot['Factor 1'] = ep_seg_vol_factors[:,0]
df_plot['Factor 2'] = ep_seg_vol_factors[:,1]
df_plot['Factor 4'] = ep_seg_vol_factors[:,3]
df_plot['Factor Sum'] = ep_seg_vol_factors[:,0] + ep_seg_vol_factors[:,1] + ep_seg_vol_factors[:,2]

df_plot = df_plot.dropna()

sns.lmplot(x='Fivesense', y='Factor 4',  data=df_plot[df_plot['Final Location']=='Temporal'])
plt.xlabel('5-sense Score', fontweight='bold')
plt.ylabel('Factor 4', fontweight='bold')

plt.figure()
sns.lmplot(x='Fivesense', y='Factor 3',  data=df_plot[df_plot['Final Location']=='Temporal'])
plt.xlabel('5-sense Score', fontweight='bold')
plt.ylabel('Factor 3', fontweight='bold')

plt.figure()
sns.lmplot(x='Fivesense', y='Factor 2',  data=df_plot[df_plot['Final Location']=='Temporal'])
plt.xlabel('5-sense Score', fontweight='bold')
plt.ylabel('Factor 2', fontweight='bold')

plt.figure()
sns.lmplot(x='Fivesense', y='Factor 1',  data=df_plot[df_plot['Final Location']=='Temporal'])
plt.xlabel('5-sense Score', fontweight='bold')
plt.ylabel('Factor 1', fontweight='bold')

plt.figure()
sns.lmplot(x='Fivesense', y='Factor Sum',  data=df_plot[df_plot['Final Location']=='Temporal'])

In [ ]:
print('Factor 1: ',stats.pearsonr(df_plot['Fivesense'],df_plot['Factor 1']))
print('Factor 2: ',stats.pearsonr(df_plot['Fivesense'],df_plot['Factor 2']))
print('Factor 3: ',stats.pearsonr(df_plot['Fivesense'],df_plot['Factor 3']))
print('Factor 4: ',stats.pearsonr(df_plot['Fivesense'],df_plot['Factor 4']))

## Figure 5 - Hierarchical clustering of atrophy factors

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list


# Perform hierarchical clustering
row_clusters = linkage(ep_seg_vol_factors, method='ward')

# Plot dendrogram to find the optimal number of clusters
fig, ax = plt.subplots(figsize=(10, 4))
dendrogram(row_clusters, ax=ax)
ax.set_title('Dendrogram')
#plt.savefig('dendrogram.pdf')
plt.show()

# Reorder data based on clustering
rows_order = leaves_list(row_clusters)
sorted_data = ep_seg_vol_factors[rows_order, :]

In [ ]:
# Plot sorted data
fig, ax = plt.subplots(figsize=(4, 8))
cax = ax.imshow(sorted_data, aspect='auto', cmap='inferno')
fig.colorbar(cax)
ax.set_title('Sorted Data')
plt.yticks(np.arange(len(rows_order)), rows_order)
plt.xticks([0,1,2,3],[1,2,3,4])
plt.xlabel('Factor Number')
#plt.savefig('sorted_factor_data_dendrogram.pdf')
plt.show()